Installing Library

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00


Importing Libraries

In [3]:
from itertools import chain

import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords', quiet=True)
# nltk.download('punkt', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


import numpy as np
import pandas as pd
import pickle
import seaborn as sns

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

from sklearn import feature_extraction
from sklearn.metrics import confusion_matrix
from sklearn import model_selection as ms
from sklearn import naive_bayes
from sklearn import preprocessing
from sklearn.metrics import classification_report

import string

Loading the Dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [5]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 9543
})

In [6]:
dataset['train']['text'][1]

'$CCL $RCL - Nomura points to bookings weakness at Carnival and Royal Caribbean https://t.co/yGjpT2ReD3'

In [7]:
for i in range(10):
  print(dataset['train']['text'][i]," Label:" ,dataset['train']['label'][i])

$BYND - JPMorgan reels in expectations on Beyond Meat https://t.co/bd0xbFGjkT  Label: 0
$CCL $RCL - Nomura points to bookings weakness at Carnival and Royal Caribbean https://t.co/yGjpT2ReD3  Label: 0
$CX - Cemex cut at Credit Suisse, J.P. Morgan on weak building outlook https://t.co/KN1g4AWFIb  Label: 0
$ESS: BTIG Research cuts to Neutral https://t.co/MCyfTsXc2N  Label: 0
$FNKO - Funko slides after Piper Jaffray PT cut https://t.co/z37IJmCQzB  Label: 0
$FTI - TechnipFMC downgraded at Berenberg but called Top Pick at Deutsche Bank https://t.co/XKcPDilIuU  Label: 0
$GM - GM loses a bull https://t.co/tdUfG5HbXy  Label: 0
$GM: Deutsche Bank cuts to Hold https://t.co/7Fv1ZiFZBS  Label: 0
$GTT: Cowen cuts to Market Perform  Label: 0
$HNHAF $HNHPD $AAPL - Trendforce cuts iPhone estimate after Foxconn delay https://t.co/rlnEwzlzzS  Label: 0


In [8]:
!git lfs install

Git LFS initialized.


In [9]:
!git clone https://huggingface.co/datasets/zeroshot/twitter-financial-news-sentiment

Cloning into 'twitter-financial-news-sentiment'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 139 (delta 0), reused 0 (delta 0), pack-reused 138
Receiving objects: 100% (139/139), 1.52 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [10]:
data = load_dataset('/content/twitter-financial-news-sentiment')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [11]:
df = pd.read_csv('/content/twitter-financial-news-sentiment/sent_train.csv')
df.head(5)

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0


Exploring the Data

In [12]:
df.head(5)

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0


In [13]:
df.tail(15)

,text,label
9528,Stocks making the biggest moves in the premark...,2
9529,Stocks making the biggest moves midday: Amazon...,2
9530,"Stocks making the biggest moves midday: Coty, ...",2
9531,Stocks making the biggest moves midday: Home D...,2
9532,Stocks making the biggest moves midday: Netfli...,2
9533,"Stocks making the biggest moves midday: Tesla,...",2
9534,"Stocks making the biggest moves midday: Tesla,...",2
9535,"Stocks making the biggest moves midday: Uber, ...",2
9536,Stocks making the biggest moves premarket: Tar...,2
9537,Stocks making the biggest moves premarket: TD ...,2


Checking for unique values

In [14]:
labels = list(df['label'].unique())
labels

[0, 1, 2]

In [15]:
df.shape

(9543, 2)

In [16]:
cleaned_text = list(df['text'].unique())
len(cleaned_text)

9543

In [17]:
df['label'].value_counts()

label
2    6178
1    1923
0    1442
Name: count, dtype: int64

In [18]:
lab_unique_val = {}

for label in labels:
    mask = df['label'] == label
    list_length = len(list(df[mask]['text'].unique()))
    lab_unique_val.update({label: list_length})

    lab_unique_val

In [19]:
lab_unique_val_sorted = {}
sorted_keys = sorted(lab_unique_val, key=lab_unique_val.get)

for w in list(reversed(sorted_keys)):
    lab_unique_val_sorted[w] = lab_unique_val[w]

# cat_unique_val_sorted
lab_unique_val_sorted

{2: 6178, 1: 1923, 0: 1442}

Dropping Duplicates

In [20]:
data = df.drop_duplicates()

In [21]:
df.shape

(9543, 2)

# Data Preprocessing

In [22]:
import re

Removing stop words and punctuations

In [23]:
stop_words = set(stopwords.words('english'))

In [24]:
regular_punct = list(string.punctuation)

In [25]:
def text_preprocessing(x):
    filtered_sentence = []
    word_tokens = word_tokenize(x)

    for w in word_tokens:
        if w not in chain(stop_words, regular_punct):
            # we make sure that all words are written in lowercase
            filtered_sentence.append(w.lower())

    # Converting a list of strings back to a string
    filtered_sentence = " ".join(filtered_sentence)
    return filtered_sentence

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
df['cleaned_text'] = df['text'].apply(text_preprocessing)

In [28]:
df['cleaned_text']

0       bynd jpmorgan reels expectations beyond meat h...
1       ccl rcl nomura points bookings weakness carniv...
2       cx cemex cut credit suisse j.p. morgan weak bu...
3       ess btig research cuts neutral https //t.co/mc...
4       fnko funko slides piper jaffray pt cut https /...
                              ...                        
9538    the week 's gainers losers stoxx europe 50 -- ...
9539    tupperware brands among consumer gainers unile...
9540    vtv therapeutics leads healthcare gainers myom...
9541                  work xpo pyx amkr among hour movers
9542                     yndx i qd oesx among tech movers
Name: cleaned_text, Length: 9543, dtype: object

In [29]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Tokenize text
    words = word_tokenize(text)

    # Remove stopwords and make lowercase
    cleaned_text = [word.lower() for word in words if word.lower() not in stop_words]

    # Join words to form the cleaned text
    cleaned_text = ' '.join(cleaned_text)

    return cleaned_text

# Apply the cleaning function to the text column
data['cleaned_text'] = data['text'].apply(clean_text)

# Show the cleaned text
data[['text', 'cleaned_text']].head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,cleaned_text
0,$BYND - JPMorgan reels in expectations on Beyo...,bynd jpmorgan reels expectations beyond meat
1,$CCL $RCL - Nomura points to bookings weakness...,ccl rcl nomura points bookings weakness carniv...
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",cx cemex cut credit suisse jp morgan weak buil...
3,$ESS: BTIG Research cuts to Neutral https://t....,ess btig research cuts neutral
4,$FNKO - Funko slides after Piper Jaffray PT cu...,fnko funko slides piper jaffray pt cut


In [30]:
counts = feature_extraction.text.CountVectorizer()
X = counts.fit_transform(df['cleaned_text'].values)
X.shape

(9543, 20673)

In [31]:
y = df['label'].values

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


In [33]:

if not isinstance(X_test, pd.DataFrame):
    X_test = pd.DataFrame(X_test.toarray())  # use toarray() if X_test is a sparse matrix

if not isinstance(y_test, pd.DataFrame):
    y_test = pd.DataFrame(y_test)

# Combine X_test and y_test back into a single DataFrame
test_data = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

# Save the test data to a CSV file
test_data.to_csv('test_data.csv', index=False)

In [34]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)
from sklearn.metrics import accuracy_score

In [35]:
y_pred = model_naive.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7727748691099476


In [36]:
pickle.dump(model_naive, open('/content/model.pk', 'wb'))

In [37]:
pickled_model = pickle.load(open('model.pk', 'rb'))

In [38]:
pickled_model.score(X_train, y_train)

0.9028334842799844

In [39]:
pickled_model.score(X_test, y_test)

0.7727748691099476

In [40]:
y_pred = pickled_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 64,  29,  58],
       [  9, 129,  73],
       [ 16,  32, 545]])

In [41]:
X_test[0]

0      0
1      0
2      0
3      0
4      0
      ..
950    0
951    0
952    0
953    0
954    0
Name: 0, Length: 955, dtype: int64